# 모듈 임포트

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import imaplib
import email
from email.header import decode_header, make_header
import re
import chardet

# 저장할 데이터 프레임 불러오기
사실 별 필요없음 만든 데이터 프레임을 바로 저장해도 됨

In [2]:
# 미리 만들어둔 빈 데이터 프레임
mail_df = pd.read_csv("./mail.csv")
mail_df.shape

(0, 4)

# 네이버 계정에 접속하기
접속 후 전체 메일함의 메일들 가져오기

In [3]:
imap = imaplib.IMAP4_SSL("imap.naver.com")

In [4]:
google_id = "caose9"
google_pw = pd.read_csv("./google_passwoed/naver_password.csv")
google_pw = list(google_pw)[0]

In [5]:
imap.login(google_id, google_pw)

('OK',
 [b'[CAPABILITY IMAP4rev1 LITERAL+ SASL-IR ID LIST-EXTENDED SPECIAL-USE UNSELECT QUOTA MOVE AUTH=PLAIN] Logged in'])

In [6]:
# 사서함 선택, 반환 데이터는 mailbox에 있는 메시지 수
imap.select("INBOX")

('OK', [b'70'])

In [7]:
# 사서함의 모든 메일의 uid 정보 가져오기
# 만약 특정 발신 메일만 선택하고 싶다면 'ALL' 대신에 '(FROM "xxxxx@naver.com")' 입력
status, messages = imap.uid('search', None, 'ALL')
status

'OK'

In [8]:
messages = messages[0].split()

# 메일함의 전체 메일들 크롤링해서 데이터프레임화

In [9]:
for i in range(len(messages)):
    recent_email = messages[i]
    
    # fetch 명령어로 메일 가져오기
    res, msg = imap.uid('fetch', recent_email, "(RFC822)")
    
    # 기본적으로 IMAP 서버에 연결하면 NONAUTH 상태, OK되어야 실행 가능
    # 최대 10번까지 시도해보기
    # 1번당 2초간 대기 총합 약 20초의 대기
    max_attempts = 10
    for attempt in range(max_attempts):
        # fetch 명령어로 메일 가져오기
        res, msg = imap.uid('fetch', recent_email, "(RFC822)")

        if res == "OK":
            break  # 정상적으로 연결되었으면 반복문을 빠져나감
        else:
            time.sleep(2)  # 2초 대기 후 다시 시도

    # OK상태시 나머지 코드 실행
    if res == "OK":

        # 사람이 읽을 수 있는 형태로 변환
        raw_readable = None
        for encoding in ['utf-8', 'euc-kr', 'cp949', 'shift-jis', 'iso-2022-jp']:
            try:
                raw_readable = msg[0][1].decode(encoding)
                break
            except UnicodeDecodeError:
                continue

        # raw_readable에서 원하는 부분만 파싱하기 위해 email 모듈을 이용해 변환
        email_message = email.message_from_string(raw_readable)

        # 보낸사람
        try:
            fr_header = make_header(decode_header(email_message.get('From')))    
            fr_header = email_message.get('From')
            decoded_fr = make_header(decode_header(fr_header))

            # 보낸사람 디코딩된 문자열을 문자열로 변환
            fr = str(decoded_fr)
            match = re.match(r'(.+?)\s*<(.+?)>', fr)
            name = ""
            email_address = ""
            if match:
                name = match.group(1).strip()
                email_address = match.group(2).strip()
            else:
                print("패턴이 매치되지 않았습니다.")
                if fr.split("@"):
                    fr_split = fr.split("@")
                    fr_split = fr_split[0] + "@" + fr_split[1] 
                    email_address = fr_split
                else :
                    name = fr

            print(i)
            print(name)
            print(email_address)

            # 메일 제목
            subject = make_header(decode_header(email_message.get('Subject')))

            # 메일 내용
            body = ""
            if email_message.is_multipart():
                for part in email_message.walk():
                    ctype = part.get_content_type()
                    cdispo = str(part.get('Content-Disposition'))
                    if ctype == 'text/plain' and 'attachment' not in cdispo:
                        body = part.get_payload(decode=True)  # decode
                        break
            else:
                body = email_message.get_payload(decode=True)

            # 내용 부분 디코딩
            if body:
                try:
                    body = body.decode('utf-8')
                except UnicodeDecodeError:
                    # UTF-8로 디코딩할 수 없는 경우 대체 문자로 처리
                    body = body.decode('utf-8', errors='replace')
            else:
                body = ""

            data = {'보낸이': [name], "메일주소" : [email_address], '제목': [subject], '내용': [body]}
            df = pd.DataFrame(data)
            mail_df = pd.concat([mail_df, df], ignore_index=True)
        except:
            pass

0
네이버 시리즈
series_noreply@webtoonscorp.com
1
" 현금이벤트 "
nbadmin2@imga1.com
2
네이버 시리즈
series_noreply@webtoonscorp.com
3
네이버 시리즈
series_noreply@webtoonscorp.com
4
네이버 시리즈
series_noreply@webtoonscorp.com
5
네이버 시리즈
series_noreply@webtoonscorp.com
6
네이버 시리즈
series_noreply@webtoonscorp.com
7
네이버 시리즈
series_noreply@webtoonscorp.com
8
💗이상형찾기💗
manager@successbizcom.com
9
네이버 시리즈
series_noreply@webtoonscorp.com
10
네이버 시리즈
series_noreply@webtoonscorp.com
11
네이버 시리즈
series_noreply@webtoonscorp.com
12
네이버 시리즈
series_noreply@webtoonscorp.com
13
네이버 시리즈
series_noreply@webtoonscorp.com
14
네이버 시리즈
series_noreply@webtoonscorp.com
15
네이버 시리즈
series_noreply@webtoonscorp.com
16
네이버 웹툰
series_noreply@webtoonscorp.com
17
네이버 웹툰
series_noreply@webtoonscorp.com
18
온라인마케터스
manager@successbizcom.com
19
" 갤럭시Z플립5 "
biznb2@nate.com
20
" 갤럭시Z플립5 "
biznb2@nate.com
21
네이버 시리즈
series_noreply@webtoonscorp.com
22
네이버 시리즈
series_noreply@webtoonscorp.com
23
네이버 시리즈
series_noreply@webtoonscorp.com
24
네이버 시리즈
series_noreply@w

In [10]:
mail_df.shape

(70, 4)

In [11]:
-------------

SyntaxError: invalid syntax (1241381794.py, line 1)

In [12]:
# 최종적 저장
mail_df.to_csv("./mail.csv", index = False)

In [23]:
# 내용 확인
pd.read_csv("./mail.csv").shape

(70, 4)

In [24]:
mail_df

,보낸이,메일주소,제목,내용
0,네이버 시리즈,series_noreply@webtoonscorp.com,[네이버 시리즈] '히든 특성 13개 ...'의 구매 내역을 안내해 드립니다.,
1,""" 현금이벤트 """,nbadmin2@imga1.com,(광고) 않바꾸면 손해! 인터넷 변경하고 현금 47만원 받아가세요~,"<table width=""100%"" border=""0"" cellspacing=""0""..."
2,네이버 시리즈,series_noreply@webtoonscorp.com,[네이버 시리즈] '아카데미 편의점으로...'의 구매 내역을 안내해 드립니다.,
3,네이버 시리즈,series_noreply@webtoonscorp.com,[네이버 시리즈] '나 혼자 탑에서 농...'의 구매 내역을 안내해 드립니다.,
4,네이버 시리즈,series_noreply@webtoonscorp.com,[네이버 시리즈] '멸망 예정인 아카데...'의 구매 내역을 안내해 드립니다.,
...,...,...,...,...
65,온라인마케터스,manager@successbizcom.com,"(광고)신규,잠재 고객유입 온라인 마케팅. CPA마케팅",[https://hiddenmailer.com/mail/gourl.php?campa...
66,네이버 시리즈,series_noreply@webtoonscorp.com,[네이버 시리즈] '아카데미 편의점으로...'의 구매 내역을 안내해 드립니다.,
67,CHEOMAD,admin@cheom-ad.kr,(광고) 하늘안과 겨울한정 라식/라섹 이벤트 시작! [ 4가지 특별혜택 ],"<html>\r\n\r\n<body>\r\n\r\n<table border=""0"" ..."
68,네이버,account_noreply@navercorp.com,2단계 인증 기능이 설정되었습니다.,\n<!-- 아웃룩용 max-width 핵 --> <!--[if (gte mso 9...
